In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import time 
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [ ]:
data_df = pd.read_csv('../data/train.csv')
train_data = data_df.values
print("Train Data Shape is: ",train_data.shape)
data_df.head()

In [ ]:
X = train_data[:, 1:]
y = train_data[:, 0]
print('Data shape: ', X.shape)
print('Label shape: ', y.shape)

In [ ]:
sc = StandardScaler()
sc.fit(X)
X_std = sc.transform(X)
print('Standard Data shape:', X_std.shape)
X_std[:5, :]

In [ ]:
# for n_com in [0.92, 0.94, 0.95, 0.96]:
#     pca = PCA(n_components=n_com)
#     pca.fit(X_std)
#     X_std_pca = pca.transform(X_std)
#     print('Standard PCA n_components: {}, Data shape: {}'.format(n_com, X_std_pca.shape))

In [ ]:
def find_SVC_hyper_param(X, y):
    param_grid = [
        {
            'C': [100, 10, 1],
            # 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            # 'degree': [3, 4, 5],
            'gamma': ['auto', 0.1, 0.01, 0.001],
            # 'tol': [1e-3, 1e-4, 1e-5],
            # 'random_state': [0, 100, 200, 500]
        },
    ]
    grid_search = GridSearchCV(SVC(), param_grid, n_jobs=-1, verbose=1, cv=5)
    grid_search.fit(X, y)
    print('best_SVC_score', grid_search.best_score_)
    print('best_SVC_param', grid_search.best_params_)
    return grid_search.best_estimator_

In [ ]:
pca = PCA(n_components=0.95)
pca.fit(X_std)
X_std_pca = pca.transform(X_std)
print('Standard PCA Data shape: ', X_std_pca.shape)

In [ ]:
start_time = time.time()
best_svc = find_SVC_hyper_param(X_std_pca, y)
elapsed_time = time.time() - start_time
print('best_SVC_estimator', best_svc)
print("Time consumed to find hyper param: ",time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, train_sizes=np.linspace(0.1, 1.0, 5), n_jobs=1, random_state=None):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, train_sizes=train_sizes, cv=cv, n_jobs=n_jobs, random_state=random_state)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")

In [ ]:
from sklearn.model_selection import ShuffleSplit, learning_curve

cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=666)
title = 'Learning Curves (SVC)'
plot_learning_curve(best_svc, title, X_std_pca, y, ylim=(0.5, 1.3), train_sizes=np.linspace(0.1, 1.0, 10), cv=cv, n_jobs=-1, random_state=666)